In [123]:
import pandas as pd
import plotly.express as px

In [124]:
df = pd.read_pickle("results.pkl")

In [125]:
# df[["LP Status", "IIS_paths", "IIS_edges", "IIS_global_budget","experiment_type"]]

In [126]:
df.columns, len(df)

(Index(['Original Distance', 'Time Taken', 'Success', 'Number of Paths',
        'Number of Edges', 'Add Times', 'Perturb Times', 'Iterations',
        'Final Distance', 'Status', 'LP Status', 'Perturbation Dict',
        'Total Perturbations', 'Global Budget Slack', 'Supporting Paths',
        'Perturbation Failure', 'graph_name', 'weights', 'experiment_type',
        'n_nodes_per_experiment', 'n_experiments', 'n_trials',
        'min_path_length', 'nodes', 'pairs', 'perturber_class', 'global_budget',
        'local_budget', 'epsilon', 'k', 'top_k', 'max_iterations', 'goal'],
       dtype='object'),
 47)

In [127]:
df["Average Add Time"] = df["Add Times"].map(lambda x: sum(x) / len(x))
df["Average Perturb Time"] = df["Perturb Times"].map(lambda x: sum(x) / len(x))

In [128]:
df.to_csv("results.csv")

In [129]:

for experiment_type in ["Single", "Sets", "Multiple Pairs"]:
    a = df[df["experiment_type"] == experiment_type]
    if a.empty: continue
    # a=pd.melt(df,id_vars=['race'],var_name='gender', value_name='value')
    fig = px.bar(a, 
                title=f"{experiment_type} Results",
                x="nodes", 
                y="Time Taken", 
                barmode="group", 
                color="perturber_class", 
                facet_col="weights", 
                # facet_row="weights", 
                pattern_shape="Success", 
                # height=1000,
                hover_data=["top_k", "Average Add Time", "Average Perturb Time", "Original Distance", "Number of Paths", "Number of Edges", "Iterations", 'Final Distance',"Status", "LP Status"],
                category_orders={"experiment_type": ["Single", "Sets", "Multiple Pairs"], 
                                "Status": ["Success"]+[s for s in df["Status"].unique() if s != "Success"],
                                "Success": [True, False]}
                )
    fig.update_xaxes(visible=False, matches=None)
    # fig.update_xaxes(visible=True)
    fig.show()

In [130]:

for experiment_type in ["Single", "Sets", "Multiple Pairs"]:
    a = df[df["experiment_type"] == experiment_type]
    if a.empty: continue
    fig = px.bar(a, 
                title=f"{experiment_type} Results",
                x="nodes", 
                y="Total Perturbations", 
                barmode="group", 
                color="perturber_class", 
                facet_col="weights", 
                # facet_row="weights", 
                pattern_shape="Success", 
                # height=1000,
                hover_data=["top_k", "Average Add Time", "Average Perturb Time", "Original Distance", "Number of Paths", "Number of Edges", "Iterations", "Final Distance", "Status"],
                category_orders={"experiment_type": ["Single", "Sets", "Multiple Pairs"], 
                                "Status": ["Success"]+[s for s in df["Status"].unique() if s != "Success"],
                                "Success": [True, False]}
                )
    fig.update_xaxes(visible=False, matches=None)
    fig.show()